# 0. Librerias

In [120]:
# Genérica
# -----------------------------------------------------------------------------
from time import time
import session_info


# Tratamientos datos
# -----------------------------------------------------------------------------
import pandas as pd
import numpy as np


# category encoders
# -----------------------------------------------------------------------------
from category_encoders import OrdinalEncoder


# scikit-learn
# -----------------------------------------------------------------------------
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import FunctionTransformer


# XGBoost
# -----------------------------------------------------------------------------
from xgboost import XGBRegressor


# Optuna
# -----------------------------------------------------------------------------
import optuna
optuna.logging.set_verbosity(optuna.logging.FATAL)


# Gráficos
# -----------------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt


session_info.show(html=False)

-----
category_encoders   2.6.1
matplotlib          3.7.2
numpy               1.23.5
optuna              3.2.0
pandas              1.5.3
seaborn             0.12.2
session_info        1.0.0
sklearn             1.3.0
xgboost             1.7.6
-----
IPython             8.14.0
jupyter_client      8.3.0
jupyter_core        5.3.1
notebook            6.5.4
-----
Python 3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]
Linux-5.15.0-82-generic-x86_64-with-glibc2.31
-----
Session information updated at 2023-09-05 13:16


In [61]:
# Funciones
# -----------------------------------------------------------------------------
def your_function(xxx):
    pass

# 1. Cargar datos

In [131]:
# Ruta
path = './data/tablon-sell-in-2.csv'


# Cargar datos
df = pd.read_csv(
    path,
    sep=';',
    dtype={
        'Epoca_Covid': 'category',
        'Anio': 'category',
        'Trimestre': 'category',
        'Mes': 'category',
        'Cliente': 'category',
        'Tipo_Cliente': 'category',
        'Distribuidor_MSM': 'category',
        'Codigo_Postal': 'category',
        'Area_Dist': 'category',
        'Zona': 'category',
        'ID_CCAA': 'category',
        'ID_Provincia': 'category',
        'Gama': 'category',
        'Formato': 'category'
    })


# Castear tipo de dato
fcols = df.select_dtypes('float').columns
icols = df.select_dtypes('integer').columns

df[fcols] = df[fcols].apply(pd.to_numeric, downcast='float')
df[icols] = df[icols].apply(pd.to_numeric, downcast='integer')


# Eliminar año 2020
filtro = (df.Anio != '2020')
# filtro = (df.Anio == '2022') & (df.Tipo_Cliente == 'CON')
# filtro = (df.Cliente == '542516')
df = df[filtro]


df.head()

,Fecha,Epoca_Covid,Anio,Trimestre,Mes,Cliente,Tipo_Cliente,Distribuidor_MSM,Codigo_Postal,Area_Dist,...,IPRI,Temperatura_Minima,Temperatura_Media,Temperatura_Maxima,Precipitacion,Sell_In_3,Sell_In_2,Sell_In_1,Sell_In,STRATIO_CREATED_DATE
0,201810,pre-covid,2018,T4,10,558843,DHC,0,43202,ZZ,...,102.220001,12.98,18.32,23.650000,275.600006,0.630000,0.320000,0.55,0.630000,2023-09-01T14:08:41.115Z
1,201810,pre-covid,2018,T4,10,558843,DHC,0,43202,ZZ,...,102.220001,12.98,18.32,23.650000,275.600006,0.550000,0.320000,0.48,0.240000,2023-09-01T14:08:41.115Z
2,201810,pre-covid,2018,T4,10,558850,CON,0,49220,Y4,...,102.620003,7.46,14.08,20.709999,19.040001,39.599998,109.800003,0.00,34.200001,2023-09-01T14:08:41.115Z
3,201810,pre-covid,2018,T4,10,558850,CON,0,49220,Y4,...,102.620003,7.46,14.08,20.709999,19.040001,46.200001,63.000000,0.00,42.000000,2023-09-01T14:08:41.115Z
4,201810,pre-covid,2018,T4,10,558850,CON,0,49220,Y4,...,102.620003,7.46,14.08,20.709999,19.040001,0.000000,11.640000,0.00,0.000000,2023-09-01T14:08:41.115Z


In [132]:
# Columnas del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1641192 entries, 0 to 1975967
Data columns (total 54 columns):
 #   Column                      Non-Null Count    Dtype   
---  ------                      --------------    -----   
 0   Fecha                       1641192 non-null  int32   
 1   Epoca_Covid                 1641192 non-null  category
 2   Anio                        1641192 non-null  category
 3   Trimestre                   1641192 non-null  category
 4   Mes                         1641192 non-null  category
 5   Cliente                     1641192 non-null  category
 6   Tipo_Cliente                1641192 non-null  category
 7   Distribuidor_MSM            1641192 non-null  category
 8   Codigo_Postal               1641192 non-null  category
 9   Area_Dist                   1641192 non-null  category
 10  Zona                        1641192 non-null  category
 11  ID_CCAA                     1641192 non-null  category
 12  ID_Provincia                1641192 non-nu

In [133]:
# Eliminar columnas
df = df.drop(columns=[
    'Fecha',
    'Anio',
    'Cliente',
    'Frontur',
    'Temperatura_Minima',
    'Temperatura_Maxima',
    'Movimientos_Entrada',
    'Movimientos_Salida',
    'Volumen_Solicitado',
    'STRATIO_CREATED_DATE'
])

In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1641192 entries, 0 to 1975967
Data columns (total 44 columns):
 #   Column                      Non-Null Count    Dtype   
---  ------                      --------------    -----   
 0   Epoca_Covid                 1641192 non-null  category
 1   Trimestre                   1641192 non-null  category
 2   Mes                         1641192 non-null  category
 3   Tipo_Cliente                1641192 non-null  category
 4   Distribuidor_MSM            1641192 non-null  category
 5   Codigo_Postal               1641192 non-null  category
 6   Area_Dist                   1641192 non-null  category
 7   Zona                        1641192 non-null  category
 8   ID_CCAA                     1641192 non-null  category
 9   ID_Provincia                1641192 non-null  category
 10  Gama                        1641192 non-null  category
 11  Formato                     1641192 non-null  category
 12  Num_PdV_CI                  1641192 non-nu

In [135]:
# Ordinal Encoder
categorical_features = [col for col in df.columns if df[col].dtype == 'category']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(df)

df=encoder.transform(df)

df.head()

,Epoca_Covid,Trimestre,Mes,Tipo_Cliente,Distribuidor_MSM,Codigo_Postal,Area_Dist,Zona,ID_CCAA,ID_Provincia,...,Movimientos_Neto,IPC,IPI,IPRI,Temperatura_Media,Precipitacion,Sell_In_3,Sell_In_2,Sell_In_1,Sell_In
0,1,1,1,1,1,1,1,1,1,1,...,53148.5,0.8,118.720001,102.220001,18.32,275.600006,0.630000,0.320000,0.55,0.630000
1,1,1,1,1,1,1,1,1,1,1,...,53148.5,0.8,118.720001,102.220001,18.32,275.600006,0.550000,0.320000,0.48,0.240000
2,1,1,1,2,1,2,2,2,2,2,...,5483.5,1.2,109.089996,102.620003,14.08,19.040001,39.599998,109.800003,0.00,34.200001
3,1,1,1,2,1,2,2,2,2,2,...,5483.5,1.2,109.089996,102.620003,14.08,19.040001,46.200001,63.000000,0.00,42.000000
4,1,1,1,2,1,2,2,2,2,2,...,5483.5,1.2,109.089996,102.620003,14.08,19.040001,0.000000,11.640000,0.00,0.000000


# 2. Modelos

## 2.1. Sin feature engineering

### 2.1.1. Modelo base

In [136]:
# Variable dependiente
target = 'Sell_In'

# Variables independientes
features = list(df.columns)
features.remove(target)

# Mostrar
print(f'Variable dependiente: {target}')
print(30*'-')
print(f'Variable independiente: {features}')

Variable dependiente: Sell_In
------------------------------
Variable independiente: ['Epoca_Covid', 'Trimestre', 'Mes', 'Tipo_Cliente', 'Distribuidor_MSM', 'Codigo_Postal', 'Area_Dist', 'Zona', 'ID_CCAA', 'ID_Provincia', 'Gama', 'Formato', 'Num_PdV_CI', 'Num_PdV_RU', 'Num_PdV_TU', 'Impactos_TV', 'Impactos_OOH', 'Impresiones_Meta', 'Impresiones_Pint', 'Impresiones_Digi', 'Sell_In_PP', 'Marketing_Nacional', 'Trade_Marketing', 'Marketing_PLV', 'Distancia_CdP', 'Dispersion_PdV', 'Volumen_Entregado_Perfecto', 'Nivel_Servicio', 'Tarifa_Eur_Hl', 'Margen_Distribuidor', 'Num_Festivos', 'Num_Festivos_AI', 'Egatur', 'Familitur', 'Movimientos_Neto', 'IPC', 'IPI', 'IPRI', 'Temperatura_Media', 'Precipitacion', 'Sell_In_3', 'Sell_In_2', 'Sell_In_1']


In [137]:
# Seed
seed = 123

# Separar conjunto de test y entrenamiento
train, test = train_test_split(df, train_size=0.95, test_size=0.05, random_state=seed)

In [113]:
# Instanciar modelo
max_depth = 8

xgbr = XGBRegressor(
    booster = 'gbtree', # Gradient Boosting Decision Tree
    learning_rate = 0.1, # Boosting learning rate, aka eta
    n_estimators = 1000, # Número de árboles
    max_depth = max_depth, # Profundidad máxima del árbol
    max_leaves = 80, # Número de hojas máximo del árbol
    colsample_bytree = 0.9, # Porcentaje de features en cada árbol
    subsample = 0.9, # Porcentaje de registros en cada árbol
    # eval_metric = 'mae',
    # reg_alpha = ???, # L1
    # reg_lamba = ???, # L2
    random_state = seed,
    n_jobs = -1,
    verbosity = 1, # logs 0 (silent), 1 (warning), 2 (info), 3 (debug).
    )

In [114]:
# Pipeline
model_pipeline = Pipeline(steps=[("xgbr", xgbr)])

In [115]:
# Medir tiempo
tiempo_inicial = time()

# Entrenar modelo
model = model_pipeline.fit(train[features], train[target])

# Medir tiempo
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial

print('')
print(f'El tiempo de ejecución es de {tiempo_ejecucion/60} minutos')


El tiempo de ejecución es de 0.035784383614857994 minutos


In [116]:
# Cálculo de métricas
train_y_real = train[target]
train_y_pred = model.predict(train[features])

test_y_real = test[target]
test_y_pred = model.predict(test[features])

metric_test_r2 = round(r2_score(test_y_real, test_y_pred), 3)
metric_adj_test_r2 = round(1-(1-metric_test_r2)*(len(test)-1)/(len(test)-len(features)-1), 3)

metric_train_mae = round(mean_absolute_error(train_y_real, train_y_pred), 3)
metric_test_mae = round(mean_absolute_error(test_y_real, test_y_pred), 3)

metric_train_rmse = round(mean_squared_error(train_y_real, train_y_pred, squared=False), 3)
metric_test_rmse = round(mean_squared_error(test_y_real, test_y_pred, squared=False), 3)

metric_train_score = round(1.15 - (metric_train_mae / train_y_real.mean()), 3)
metric_test_score = round(1.15 - (metric_test_mae / test_y_real.mean()), 3)


# Mostrar
print('R2 Score')
print(50*'-')
print(f'Test R2: {metric_test_r2}')
print(f'Test R2 Adj: {metric_adj_test_r2}')
print('')

print('Score')
print(50*'-')
print(f'Train: {metric_train_score}')
print(f'Test: {metric_test_score}')
print('')

print('MAE')
print(50*'-')
print(f'Train: {metric_train_mae}')
print(f'Test: {metric_test_mae}')
print('')

print('RMSE')
print(50*'-')
print(f'Train: {metric_train_rmse}')
print(f'Test: {metric_test_rmse}')
print('')

R2 Score
--------------------------------------------------
Test R2: 0.984
Test R2 Adj: 0.973

Score
--------------------------------------------------
Train: 1.15
Test: 1.026

MAE
--------------------------------------------------
Train: 0.0010000000474974513
Test: 35.5

RMSE
--------------------------------------------------
Train: 0.0010000000474974513
Test: 110.38899993896484



In [117]:
feature_imp = pd.DataFrame(sorted(zip(features, model[0].feature_importances_)), columns=['Feature', 'Value'])
feature_imp.sort_values(by=['Value'], ascending=False)

,Feature,Value
32,Sell_In_1,0.611652
34,Sell_In_3,0.197183
8,Formato,0.117036
28,Num_PdV_CI,0.032043
5,Egatur,0.007555
33,Sell_In_2,0.005229
21,Marketing_Nacional,0.004335
25,Nivel_Servicio,0.004006
39,Trade_Marketing,0.003624
35,Sell_In_PP,0.003293


### 2.1.2. Modelo optimizado

In [118]:
# Variable dependiente
target = 'Sell_In'

X = df.drop(target, axis=1)
y = df[target]

In [126]:
# Intanciar regresor base
regressor_obj = XGBRegressor(
    booster = 'gbtree',
    n_estimators=100,
    n_jobs=-1,
    random_state=seed
)
    
# Sugerir parámetros a optimizar
param_distributions = {
    "learning_rate": optuna.distributions.FloatDistribution(1e-8, 1.0, log=True),
    "max_depth": optuna.distributions.IntDistribution(4, 10, step=1, log=False),
    "max_leaves": optuna.distributions.IntDistribution(16, 120, step=5, log=False),
    "colsample_bytree": optuna.distributions.FloatDistribution(0.5, 1.0, step=0.1, log=False),
    "subsample": optuna.distributions.FloatDistribution(0.5, 1.0, step=0.1, log=False),
    "reg_alpha": optuna.distributions.FloatDistribution(1e-8, 1.0, log=True),
    "reg_lambda": optuna.distributions.FloatDistribution(1e-8, 1.0, log=True),
}

# Instanciar optimizador
optuna_search = optuna.integration.OptunaSearchCV(
    regressor_obj,
    param_distributions,
    # Cross Validation
    cv=5,
    # Number of trials. This trades off runtime vs quality of the solution.
    n_trials=200,
    # Time limit in seconds for the search of appropriate models. This trades off runtime vs quality of the solution.
    timeout=1200,
    verbose=-1
)

/home/gmachin/.cache/pypoetry/virtualenvs/msm-sell-in-prediction-unVKpS6j-py3.10/lib/python3.10/site-packages/optuna/distributions.py:700: UserWarning: The distribution is specified by [16, 120] and step=5, but the range is not divisible by `step`. It will be replaced by [16, 116].
  warnings.warn(
/tmp/ipykernel_34088/960894016.py:21: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(


In [127]:
# Medir tiempo
tiempo_inicial = time()


# Ejecutar optimizador
optuna_search.fit(X, y)


# Medir tiempo
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial

print('')
print(f'El tiempo de ejecución es de {tiempo_ejecucion/60} minutos')


El tiempo de ejecución es de 3.4777498642603555 minutos


In [128]:
# Mostrar score_r2 y los mejores hiperparámetros
print("Best score:", optuna_search.best_score_)
print("Best parameters:", optuna_search.best_params_)

Best score: 0.9483258583953669
Best parameters: {'learning_rate': 0.06325588715490191, 'max_depth': 10, 'max_leaves': 76, 'colsample_bytree': 0.5, 'subsample': 0.6, 'reg_alpha': 1.3190491111761998e-07, 'reg_lambda': 2.620778874518851e-07}


In [130]:
# Instanciar modelo
xgbr = XGBRegressor(
    booster = 'gbtree',
    learning_rate = optuna_search.best_params_['learning_rate'],
    n_estimators = 100,
    max_depth = optuna_search.best_params_['max_depth'],
    max_leaves = optuna_search.best_params_['max_leaves'],
    colsample_bytree = optuna_search.best_params_['colsample_bytree'],
    subsample = optuna_search.best_params_['subsample'],
    reg_alpha = optuna_search.best_params_['reg_alpha'],
    reg_lambda = optuna_search.best_params_['reg_lambda'],
    random_state = seed,
    n_jobs = -1,
    verbosity = 1, # logs 0 (silent), 1 (warning), 2 (info), 3 (debug).
    )

# Pipeline
model_pipeline = Pipeline(steps=[("xgbr", xgbr)])

# Entrenar modelo
model = model_pipeline.fit(train[features], train[target])


# Cálculo de métricas
train_y_real = train[target]
train_y_pred = model.predict(train[features])

test_y_real = test[target]
test_y_pred = model.predict(test[features])

metric_test_r2 = round(r2_score(test_y_real, test_y_pred), 3)
metric_adj_test_r2 = round(1-(1-metric_test_r2)*(len(test)-1)/(len(test)-len(features)-1), 3)

metric_train_mae = round(mean_absolute_error(train_y_real, train_y_pred), 3)
metric_test_mae = round(mean_absolute_error(test_y_real, test_y_pred), 3)

metric_train_rmse = round(mean_squared_error(train_y_real, train_y_pred, squared=False), 3)
metric_test_rmse = round(mean_squared_error(test_y_real, test_y_pred, squared=False), 3)

metric_train_score = round(1.15 - (metric_train_mae / train_y_real.mean()), 3)
metric_test_score = round(1.15 - (metric_test_mae / test_y_real.mean()), 3)

# Mostrar
print('R2 Score')
print(50*'-')
print(f'Test R2: {metric_test_r2}')
print(f'Test R2 Adj: {metric_adj_test_r2}')
print('')

print('Score')
print(50*'-')
print(f'Train: {metric_train_score}')
print(f'Test: {metric_test_score}')
print('')

print('MAE')
print(50*'-')
print(f'Train: {metric_train_mae}')
print(f'Test: {metric_test_mae}')
print('')

print('RMSE')
print(50*'-')
print(f'Train: {metric_train_rmse}')
print(f'Test: {metric_test_rmse}')
print('')

R2 Score
--------------------------------------------------
Test R2: 0.983
Test R2 Adj: 0.971

Score
--------------------------------------------------
Train: 1.133
Test: 1.01

MAE
--------------------------------------------------
Train: 4.821000099182129
Test: 40.250999450683594

RMSE
--------------------------------------------------
Train: 8.227999687194824
Test: 113.74199676513672



## 2.2. Eliminando variables correlacionadas

In [92]:
# Ruta
path = './data/tablon-sell-in-2.csv'


# Cargar datos
df = pd.read_csv(
    path,
    sep=';',
    dtype={
        'Epoca_Covid': 'category',
        'Anio': 'category',
        'Trimestre': 'category',
        'Mes': 'category',
        'Cliente': 'category',
        'Tipo_Cliente': 'category',
        'Distribuidor_MSM': 'category',
        'Codigo_Postal': 'category',
        'Area_Dist': 'category',
        'Zona': 'category',
        'ID_CCAA': 'category',
        'ID_Provincia': 'category',
        'Gama': 'category',
        'Formato': 'category'
    })


# Castear tipo de dato
fcols = df.select_dtypes('float').columns
icols = df.select_dtypes('integer').columns

df[fcols] = df[fcols].apply(pd.to_numeric, downcast='float')
df[icols] = df[icols].apply(pd.to_numeric, downcast='integer')


# Eliminar año 2020
filtro = (df.Anio != '2020')
# filtro = (df.Anio == '2022') & (df.Tipo_Cliente == 'CON')
# filtro = (df.Cliente == '542516')
df = df[filtro]


df.head()

,Fecha,Epoca_Covid,Anio,Trimestre,Mes,Cliente,Tipo_Cliente,Distribuidor_MSM,Codigo_Postal,Area_Dist,...,IPRI,Temperatura_Minima,Temperatura_Media,Temperatura_Maxima,Precipitacion,Sell_In_3,Sell_In_2,Sell_In_1,Sell_In,STRATIO_CREATED_DATE
0,201810,pre-covid,2018,T4,10,558843,DHC,0,43202,ZZ,...,102.220001,12.98,18.32,23.650000,275.600006,0.630000,0.320000,0.55,0.630000,2023-09-01T14:08:41.115Z
1,201810,pre-covid,2018,T4,10,558843,DHC,0,43202,ZZ,...,102.220001,12.98,18.32,23.650000,275.600006,0.550000,0.320000,0.48,0.240000,2023-09-01T14:08:41.115Z
2,201810,pre-covid,2018,T4,10,558850,CON,0,49220,Y4,...,102.620003,7.46,14.08,20.709999,19.040001,39.599998,109.800003,0.00,34.200001,2023-09-01T14:08:41.115Z
3,201810,pre-covid,2018,T4,10,558850,CON,0,49220,Y4,...,102.620003,7.46,14.08,20.709999,19.040001,46.200001,63.000000,0.00,42.000000,2023-09-01T14:08:41.115Z
4,201810,pre-covid,2018,T4,10,558850,CON,0,49220,Y4,...,102.620003,7.46,14.08,20.709999,19.040001,0.000000,11.640000,0.00,0.000000,2023-09-01T14:08:41.115Z


In [93]:
# Columnas del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1641192 entries, 0 to 1975967
Data columns (total 54 columns):
 #   Column                      Non-Null Count    Dtype   
---  ------                      --------------    -----   
 0   Fecha                       1641192 non-null  int32   
 1   Epoca_Covid                 1641192 non-null  category
 2   Anio                        1641192 non-null  category
 3   Trimestre                   1641192 non-null  category
 4   Mes                         1641192 non-null  category
 5   Cliente                     1641192 non-null  category
 6   Tipo_Cliente                1641192 non-null  category
 7   Distribuidor_MSM            1641192 non-null  category
 8   Codigo_Postal               1641192 non-null  category
 9   Area_Dist                   1641192 non-null  category
 10  Zona                        1641192 non-null  category
 11  ID_CCAA                     1641192 non-null  category
 12  ID_Provincia                1641192 non-nu

In [94]:
# Eliminar columnas
df = df.drop(columns=[
    'Fecha',
    'Anio',
    'Cliente',
    'Codigo_Postal',
    'Frontur',
    'Temperatura_Minima',
    'Temperatura_Maxima',
    'Movimientos_Entrada',
    'Movimientos_Salida',
    'Volumen_Solicitado',
    'Volumen_Entregado_Perfecto',
    'Sell_In_PP',
    'Sell_In_3',
    'Sell_In_2',
    'Sell_In_1',
    #'Marketing_Nacional',
    #'Trade_Marketing',
    #'Marketing_PLV',
    'STRATIO_CREATED_DATE'
])

# Eliminar registros con NaN
df = df.dropna()

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1641190 entries, 0 to 1975967
Data columns (total 38 columns):
 #   Column               Non-Null Count    Dtype   
---  ------               --------------    -----   
 0   Epoca_Covid          1641190 non-null  category
 1   Trimestre            1641190 non-null  category
 2   Mes                  1641190 non-null  category
 3   Tipo_Cliente         1641190 non-null  category
 4   Distribuidor_MSM     1641190 non-null  category
 5   Area_Dist            1641190 non-null  category
 6   Zona                 1641190 non-null  category
 7   ID_CCAA              1641190 non-null  category
 8   ID_Provincia         1641190 non-null  category
 9   Gama                 1641190 non-null  category
 10  Formato              1641190 non-null  category
 11  Num_PdV_CI           1641190 non-null  float32 
 12  Num_PdV_RU           1641190 non-null  float32 
 13  Num_PdV_TU           1641190 non-null  float32 
 14  Impactos_TV          1641190 non-n

In [96]:
# Ordinal Encoder
categorical_features = [col for col in df.columns if df[col].dtype == 'category']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(df)

df=encoder.transform(df)

df.head()

,Epoca_Covid,Trimestre,Mes,Tipo_Cliente,Distribuidor_MSM,Area_Dist,Zona,ID_CCAA,ID_Provincia,Gama,...,Num_Festivos_AI,Egatur,Familitur,Movimientos_Neto,IPC,IPI,IPRI,Temperatura_Media,Precipitacion,Sell_In
0,1,1,1,1,1,1,1,1,1,1,...,0,1710.040039,0.0,53148.5,0.8,118.720001,102.220001,18.32,275.600006,0.630000
1,1,1,1,1,1,1,1,1,1,1,...,0,1710.040039,0.0,53148.5,0.8,118.720001,102.220001,18.32,275.600006,0.240000
2,1,1,1,2,1,2,2,2,2,1,...,0,96.709999,0.0,5483.5,1.2,109.089996,102.620003,14.08,19.040001,34.200001
3,1,1,1,2,1,2,2,2,2,1,...,0,96.709999,0.0,5483.5,1.2,109.089996,102.620003,14.08,19.040001,42.000000
4,1,1,1,2,1,2,2,2,2,1,...,0,96.709999,0.0,5483.5,1.2,109.089996,102.620003,14.08,19.040001,0.000000


### 2.2.1. Modelo base

In [97]:
# Variable dependiente
target = 'Sell_In'

# Variables independientes
features = list(df.columns)
features.remove(target)

# Mostrar
print(f'Variable dependiente: {target}')
print(30*'-')
print(f'Variable independiente: {features}')

Variable dependiente: Sell_In
------------------------------
Variable independiente: ['Epoca_Covid', 'Trimestre', 'Mes', 'Tipo_Cliente', 'Distribuidor_MSM', 'Area_Dist', 'Zona', 'ID_CCAA', 'ID_Provincia', 'Gama', 'Formato', 'Num_PdV_CI', 'Num_PdV_RU', 'Num_PdV_TU', 'Impactos_TV', 'Impactos_OOH', 'Impresiones_Meta', 'Impresiones_Pint', 'Impresiones_Digi', 'Marketing_Nacional', 'Trade_Marketing', 'Marketing_PLV', 'Distancia_CdP', 'Dispersion_PdV', 'Nivel_Servicio', 'Tarifa_Eur_Hl', 'Margen_Distribuidor', 'Num_Festivos', 'Num_Festivos_AI', 'Egatur', 'Familitur', 'Movimientos_Neto', 'IPC', 'IPI', 'IPRI', 'Temperatura_Media', 'Precipitacion']


In [98]:
# Seed
seed = 123

# Separar conjunto de test y entrenamiento
train, test = train_test_split(df, train_size=0.95, test_size=0.05, random_state=seed)

In [99]:
# Instanciar modelo
xgbr = XGBRegressor(
    booster = 'gbtree', # Gradient Boosting Decision Tree
    learning_rate = 0.1, # Boosting learning rate, aka eta
    n_estimators = 1000, # Número de árboles
    max_depth = max_depth, # Profundidad máxima del árbol
    max_leaves = 80, # Número de hojas máximo del árbol
    colsample_bytree = 0.9, # Porcentaje de features en cada árbol
    subsample = 0.9, # Porcentaje de registros en cada árbol
    # eval_metric = 'mae',
    # reg_alpha = ???, # L1
    # reg_lamba = ???, # L2
    random_state = seed,
    n_jobs = -1,
    verbosity = 1, # logs 0 (silent), 1 (warning), 2 (info), 3 (debug).
    )

In [100]:
# Pipeline
model_pipeline = Pipeline(steps=[("xgbr", xgbr)])

In [101]:
# Medir tiempo
tiempo_inicial = time()

# Entrenar modelo
model = model_pipeline.fit(train[features], train[target])

# Medir tiempo
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial

print('')
print(f'El tiempo de ejecución es de {tiempo_ejecucion/60} minutos')


El tiempo de ejecución es de 19.907983541488647 minutos


In [102]:
# Cálculo de métricas
train_y_real = train[target]
train_y_pred = model.predict(train[features])

test_y_real = test[target]
test_y_pred = model.predict(test[features])

metric_test_r2 = round(r2_score(test_y_real, test_y_pred), 3)
metric_adj_test_r2 = round(1-(1-metric_test_r2)*(len(test)-1)/(len(test)-len(features)-1), 3)

metric_train_mae = round(mean_absolute_error(train_y_real, train_y_pred), 3)
metric_test_mae = round(mean_absolute_error(test_y_real, test_y_pred), 3)

metric_train_rmse = round(mean_squared_error(train_y_real, train_y_pred, squared=False), 3)
metric_test_rmse = round(mean_squared_error(test_y_real, test_y_pred, squared=False), 3)

metric_train_score = round(1.15 - (metric_train_mae / train_y_real.mean()), 3)
metric_test_score = round(1.15 - (metric_test_mae / test_y_real.mean()), 3)


# Mostrar
print('R2 Score')
print(50*'-')
print(f'Test R2: {metric_test_r2}')
print(f'Test R2 Adj: {metric_adj_test_r2}')
print('')

print('Score')
print(50*'-')
print(f'Train: {metric_train_score}')
print(f'Test: {metric_test_score}')
print('')

print('MAE')
print(50*'-')
print(f'Train: {metric_train_mae}')
print(f'Test: {metric_test_mae}')
print('')

print('RMSE')
print(50*'-')
print(f'Train: {metric_train_rmse}')
print(f'Test: {metric_test_rmse}')
print('')

R2 Score
--------------------------------------------------
Test R2: 0.909
Test R2 Adj: 0.909

Score
--------------------------------------------------
Train: 0.982
Test: 0.849

MAE
--------------------------------------------------
Train: 3.236999988555908
Test: 5.426000118255615

RMSE
--------------------------------------------------
Train: 11.706999778747559
Test: 41.0620002746582



In [103]:
feature_imp = pd.DataFrame(sorted(zip(features, model[0].feature_importances_)), columns=['Feature', 'Value'])
feature_imp.sort_values(by=['Value'], ascending=False)

,Feature,Value
33,Tipo_Cliente,0.243936
34,Trade_Marketing,0.156916
20,Marketing_Nacional,0.106778
21,Marketing_PLV,0.053173
27,Num_PdV_CI,0.048544
29,Num_PdV_TU,0.040188
31,Tarifa_Eur_Hl,0.036570
36,Zona,0.031631
19,Margen_Distribuidor,0.025507
0,Area_Dist,0.018452
